In [21]:
import os
import torch
import random
import math
from torch import nn
from torch.nn.modules import activation
import torch.nn.functional as F
import torchvision
import shutil
import numpy as np
from PIL import Image
import time
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix

In [22]:
num_filters = 64
learning_rate = 0.001
num_epochs = 50
use_cuda = True
early_stop = 3
batch_size = 32
small_kernel = 1
medium_kernel = 3
large_kernel = 5
kernel_set = [[1, 3, 5]]
min_delta = -0.025
min_epochs = 15
num_workers = 8

In [27]:
class MSCB(nn.Module):
    def __init__(self, small_kernel, medium_kernel, large_kernel, num_filters):
        super(MSCB, self).__init__()
        self.name = "MSCB"

        # Define Small Path
        self.convS = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = small_kernel, padding = 'same')
        self.MPoolS = nn.MaxPool1d(kernel_size = small_kernel, stride = 5, padding = int(small_kernel/2 - 1))
        
        # Define Medium Path
        self.convM = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = medium_kernel, padding = 'same')
        self.MPoolM = nn.MaxPool1d(kernel_size = medium_kernel, stride = 5, padding = int(medium_kernel/2 - 1))
        
        # Define Large Path
        self.convL = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = large_kernel, padding = 'same')
        self.MPoolL = nn.MaxPool1d(kernel_size = large_kernel, stride = 5, padding = int(large_kernel/2 - 1))
        
        # MPool first
        self.MPool = nn.MaxPool1d(kernel_size = 3, stride = 5)
        self.conv = nn.Conv1d(in_channels = 56, out_channels = 128, kernel_size = 24, padding = 'same')
        
        # Post Concatenation
        self.conv2 = nn.Conv1d(in_channels = 3*num_filters + 128, out_channels = 64, kernel_size = 112, padding = 'same')
        self.MPool2 = nn.MaxPool1d(kernel_size = 3, stride = 5)
        self.fc1 = nn.Linear(in_features = 3840, out_features = 400)
        self.Dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(in_features = 400, out_features = 1024)
        self.fc3 = nn.Linear(in_features = 1024, out_features = 3)

    def forward(self, x):
        ### Feature Learning Head
        
        # Reshape Tensor
        x = torch.moveaxis(x,2,1)
        x = torch.tensor(x, dtype=torch.float32)

        # Parrallel Convolution Pathways
        x_S = self.MPoolS(F.relu(self.convS(x)))
        x_M = self.MPoolM(F.relu(self.convM(x)))
        x_L = self.MPoolL(F.relu(self.convL(x)))
        x_O = F.relu(self.conv(self.MPool(x)))
        
        # Post Concatenation
        x = torch.cat((x_S,x_M,x_L,x_O),1)
        x = self.MPool2(F.relu(self.conv2(x)))
        
        print(x.shape())

        # Flattening
        x = x.view(-1,3840)

        #Classification Head
        x = F.relu(self.fc1((x)))
        x = self.Dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [28]:
n_filters = 64
small_kernel = 1
medium_kernel = 3
large_kernel = 5
one_CNN = MSCB(small_kernel, medium_kernel, large_kernel, n_filters)
pytorch_total_params = sum(p.numel() for p in one_CNN.parameters())
print(pytorch_total_params)

4448531


In [17]:
class MSCB(nn.Module):
    def __init__(self, small_kernel, medium_kernel, large_kernel, num_filters):
        super(MSCB, self).__init__()
        self.name = "MSCB"

        # Define Small Path
        self.convS = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = small_kernel, padding = 'same')
        self.MPoolS = nn.MaxPool1d(kernel_size = small_kernel, stride = 5, padding = int(small_kernel/2 - 1))
        
        # Define Medium Path
        self.convM = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = medium_kernel, padding = 'same')
        self.MPoolM = nn.MaxPool1d(kernel_size = medium_kernel, stride = 5, padding = int(medium_kernel/2 - 1))
        
        # Define Large Path
        self.convL = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = large_kernel, padding = 'same')
        self.MPoolL = nn.MaxPool1d(kernel_size = large_kernel, stride = 5, padding = int(large_kernel/2 - 1))
        
        # MPool first
        self.MPool = nn.MaxPool1d(kernel_size = 3, stride = 5)
        self.conv = nn.Conv1d(in_channels = 56, out_channels = 128, kernel_size = 24, padding = 'same')
        
        # Post Concatenation
        self.conv2 = nn.Conv1d(in_channels = 3*num_filters + 128, out_channels = 64, kernel_size = 112, padding = 'same')
        self.MPool2 = nn.MaxPool1d(kernel_size = 3, stride = 5)
        self.fc1 = nn.Linear(in_features = 256, out_features = 400)
        self.Dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(in_features = 400, out_features = 1024)
        self.fc3 = nn.Linear(in_features = 1024, out_features = 3)

    def forward(self, x):
        ### Feature Learning Head
        
        # Reshape Tensor
        x = torch.moveaxis(x,2,1)
        x = torch.tensor(x, dtype=torch.float32)

        # Parrallel Convolution Pathways
        x_S = self.MPoolS(F.relu(self.convS(x)))
        x_M = self.MPoolM(F.relu(self.convM(x)))
        x_L = self.MPoolL(F.relu(self.convL(x)))
        x_O = F.relu(self.conv(self.MPool(x)))
        
        # Post Concatenation
        x = torch.cat((x_S,x_M,x_L,x_O),1)
        x = self.MPool2(F.relu(self.conv2(x)))
        
        print(x.shape)

        # Flattening
        x = x.view(-1,256)

        #Classification Head
        x = F.relu(self.fc1((x)))
        x = self.Dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [18]:
n_filters = 64
small_kernel = 1
medium_kernel = 3
large_kernel = 5

one_CNN = MSCB(small_kernel, medium_kernel, large_kernel, n_filters)
# state = torch.load("M:\Peripheral Nerve Studies\MCC Projects\Aseem G\Models\Pytorch\model_checkpoints\Rat 2\September 17 Kernels 1_3_5\Rat 2kernels_1_3_5_Fold_2_final", map_location=torch.device('cpu'))
# one_CNN.load_state_dict(state)

In [19]:
x = torch.randn(1,100,56)

In [20]:
one_CNN(x)

torch.Size([1, 64, 4])


C:\Users\GillA\AppData\Local\Temp\ipykernel_12484\610203947.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)


tensor([[ 0.0153, -0.0234,  0.0368]], grad_fn=<AddmmBackward0>)